In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

sample_data


In [ ]:
!ls drive

ls: cannot access 'drive': No such file or directory


In [ ]:
%cd '/content/drive/MyDrive/GraphDTA-master'

[Errno 2] No such file or directory: '/content/drive/MyDrive/GraphDTA-master'
/content


# New Section

In [ ]:
import torch

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=baa13a3ab3a7a3d48f0919c339f86e1426e34ad4778720430e2948ccb4689fe8
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [ ]:
!pip install torch torchvision numpy pandas scikit-learn tqdm matplotlib seaborn

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install dgl

In [ ]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
import torch
import torch.nn as nn
from models.gat import GATNet
from models.gat_gcn import GAT_GATNet
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *
import matplotlib.pyplot as plt

In [ ]:

#import dataloader as dataloader
# training function at each epoch
def train(model, device, train_loader, optimizer, epoch):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, data.y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                            batch_idx * len(data.x),
                                            len(train_loader.dataset),
                                            100. * batch_idx / len(train_loader),
                                            loss.item()))

def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
    return total_labels.numpy().flatten(),total_preds.numpy().flatten()

In [ ]:
!pip install Tensorflow

In [ ]:
!pip install Scikit-learn
!pip install Matplotlib Seaborn
!pip install subword_nmt periodictable tensorflow_addons


In [ ]:
import tensorflow as tf
import glob
import json
import itertools
import periodictable as pt
import re
from itertools import chain
from operator import itemgetter
from subword_nmt.apply_bpe import BPE
import codecs

# create data.**py**

In [ ]:
import pandas as pd
import numpy as np
import json, pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
from utils import *
import sys, os
from random import shuffle
from models.gat import GATNet
from models.gat_gcn import GAT_GATNet
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *


def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),
                                          ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As',
                                           'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se',
                                           'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                                           'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    [atom.GetIsAromatic()])


def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)

    c_size = mol.GetNumAtoms()

    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append(feature / sum(feature))

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])

    return c_size, features, edge_index

    drugs = []
    compound_iso_smiles = []
    for dt_name in ['kiba', 'davis']:
        opts = ['train', 'test']
        for opt in opts:
            df = pd.read_csv('data/' + dt_name + '_' + opt + '.csv')
            compound_iso_smiles += list(df['compound_iso_smiles'])
    compound_iso_smiles = set(compound_iso_smiles)
    smile_graph = {}
    for smile in compound_iso_smiles:
        g = smile_to_graph(smile)
        smile_graph[smile] = g


#####from DTITR#####






####FROM MOLTRANS####

class BIN_Interaction_Flat(nn.Sequential):
    '''
        Interaction Network with 2D interaction map
    '''

    def __init__(self, **config):
        super(BIN_Interaction_Flat, self).__init__()
        # self.max_d = config['max_drug_seq']
        self.max_p = config['max_protein_seq']
        self.emb_size = config['emb_size']
        self.dropout_rate = config['dropout_rate']

    def forward(self, d, p, d_mask, p_mask):
        # ex_d_mask = d_mask.unsqueeze(1).unsqueeze(2)
        ex_p_mask = p_mask.unsqueeze(1).unsqueeze(2)

        # ex_d_mask = (1.0 - ex_d_mask) * -10000.0
        ex_p_mask = (1.0 - ex_p_mask) * -10000.0

        # d_emb = self.demb(d) # batch_size x seq_length x embed_size
        p_emb = self.pemb(p)

        # set output_all_encoded_layers be false, to obtain the last layer hidden states only...

        # d_encoded_layers = self.d_encoder(d_emb.float(), ex_d_mask.float())
        # print(d_encoded_layers.shape)
        #p_encoded_layers = self.p_encoder(p_emb.float(), ex_p_mask.float())
        # print(p_encoded_layers.shape)

        # repeat to have the same tensor size for aggregation
        # d_aug = torch.unsqueeze(d_encoded_layers, 2).repeat(1, 1, self.max_p, 1) # repeat along protein size
        #sp_aug = torch.unsqueeze(p_encoded_layers, 1).repeat(1, self.max_d, 1, 1)  # repeat along drug size



        ##
        # specialized embedding with positional one
        # self.demb = Embeddings(self.input_dim_drug, self.emb_size, self.max_d, self.dropout_rate)
        self.pemb = Embeddings(self.input_dim_target, self.emb_size, self.max_p, self.dropout_rate)

        # self.d_encoder = Encoder_MultipleLayers(self.n_layer, self.hidden_size, self.intermediate_size, self.num_attention_heads, self.attention_probs_dropout_prob, self.hidden_dropout_prob)
        #self.p_encoder = Encoder_MultipleLayers(self.n_layer, self.hidden_size, self.intermediate_size,
                                                ##self.hidden_dropout_prob)

        self.icnn = nn.Conv2d(1, 3, 3, padding=0)

        self.decoder = nn.Sequential(
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(True),

            nn.BatchNorm1d(512),
            nn.Linear(512, 64),
            nn.ReLU(True),

            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(True),

            # output layer
            nn.Linear(32, 1)
        )


class Embeddings(nn.Module):
    """Construct the embeddings from protein/target, position embeddings.
    """

    def __init__(self, vocab_size, hidden_size, max_position_size, dropout_rate):
        super(Embeddings, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = nn.Embedding(max_position_size, hidden_size)

        #self.LayerNorm = LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_ids):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        embeddings = words_embeddings + position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings





In [ ]:
dataset = 'davis'
#kiba
modeling = [GAT_GATNet]
model_st = modeling[0].__name__

cuda_name = "cuda:0"
# if len(sys.argv)>3:
#     cuda_name = "cuda:" + str(int(sys.argv[3]))
# print('cuda_name:', cuda_name)

TRAIN_BATCH_SIZE = 512
TEST_BATCH_SIZE = 512
LR = 0.0005
LOG_INTERVAL = 20
NUM_EPOCHS = 2

print('Learning rate: ', LR)
print('Epochs: ', NUM_EPOCHS)

# Main program: iterate over different datasets
# for dataset in dataset:
print('\nrunning on ', model_st + '_' + dataset )
processed_data_file_train = 'data/processed/' + dataset + '_train.pt'
processed_data_file_test = 'data/processed/' + dataset + '_test.pt'
if ((not os.path.isfile(processed_data_file_train)) or \
 (not os.path.isfile(processed_data_file_test))):
    print('please run create_data.py to prepare data in pytorch format!')
else:
    train_data = TestbedDataset(root='data', dataset=dataset+'_train')
    test_data = TestbedDataset(root='data', dataset=dataset+'_test')

    # make data PyTorch mini-batch processing ready
    train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

# training the model
    device = torch.device(cuda_name if torch.cuda.is_available() else "cpu")
    #model = modeling
    for modeling in [GAT_GATNet]:
      model = modeling().to(device)
    loss_fn = nn.MSELoss()

    # model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    best_mse = 0.24929152
    best_ci =  0.8811023101087032
    best_epoch = 853
    model_file_name = 'model_' + model_st + '_' + dataset +  '.model'
    result_file_name = 'result_' + model_st + '_' + dataset +  '.csv'
    model.load_state_dict(torch.\
    load('/content/drive/MyDrive/GraphDTA-master/model_GAT_GATNet_davis.model',\
         map_location=torch.device('cpu'))
    )
     #map_location=torch.device('cpu'))) # load the state_dict on CPU
    # model.train()

    mse_list = []
    ci_list = []
    for epoch in range(NUM_EPOCHS):
      torch.cuda.empty_cache()
      train(model, device, train_loader, optimizer, epoch+1)
      G,P = predicting(model, device, test_loader)
      ret = [rmse(G,P),mse(G,P),pearson(G,P),spearman(G,P),ci(G,P)]


      mse_list.append(ret[1])
      ci_list.append(ret[-1])
      if ret[1]<best_mse:
          torch.save(model.state_dict(), \
                '/content/drive/MyDrive/GraphDTA-master/model_GAT_GATNet_davis.model')
          with open(result_file_name,'w') as f:
              f.write(','.join(map(str,ret)))
          best_epoch = epoch+1
          best_mse = ret[1]
          best_ci = ret[-1]


          print('rmse improved at epoch ', best_epoch, '; best_mse,best_ci:',\
                best_mse,best_ci,model_st,dataset)

      else:
          print(ret[1],'No improvement since epoch ', best_epoch,\
                '; best_mse,best_ci:', best_mse,best_ci,model_st,dataset)




Learning rate:  0.0005
Epochs:  2

running on  GAT_GATNet_davis
Pre-processed data found: data/processed/davis_train.pt, loading ...
Pre-processed data found: data/processed/davis_test.pt, loading ...


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Training on 25046 samples...


/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "


Train epoch: 1 [0/25046 (0%)]	Loss: 0.107014
Train epoch: 1 [329680/25046 (41%)]	Loss: 0.135768
Train epoch: 1 [657720/25046 (82%)]	Loss: 0.093029
Make prediction for 5010 samples...
rmse improved at epoch  1 ; best_mse,best_ci: 0.24873847 0.8773788627588337 GAT_GATNet davis
Training on 25046 samples...
Train epoch: 2 [0/25046 (0%)]	Loss: 0.072308
Train epoch: 2 [331360/25046 (41%)]	Loss: 0.098862
Train epoch: 2 [659080/25046 (82%)]	Loss: 0.069238
Make prediction for 5010 samples...
0.2679761 No improvement since epoch  1 ; best_mse,best_ci: 0.24873847 0.8773788627588337 GAT_GATNet davis
